# Traffic Sign Indentification

In [4]:
!pip install -r setup_lib.txt

In [5]:
import os
import pandas as pd
from imageio import imread
import math
import numpy as np
import cv2
import tensorflow
import keras
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential

In [12]:
data_path = 'GTSRB/Final_Training/Images'
pixels = []
labels = []
# Loop qua các thư mục trong thư mục Images
for dir in os.listdir(data_path):
    # Bỏ qua file .DS_Store của máy MacOS
    if dir == '.DS_Store':
        continue

    # Đọc file csv để lấy thông tin về ảnh
    class_dir = os.path.join(data_path, dir)
    info_file = pd.read_csv(os.path.join(class_dir, "GT-" + dir + '.csv'), sep=';')

    # Lăp trong file
    for row in info_file.iterrows():
        # Đọc ảnh
        pixel = imread(os.path.join(class_dir, row[1].Filename))
        # Trích phần ROI theo thông tin trong file csv
        pixel = pixel[row[1]['Roi.X1']:row[1]['Roi.X2'], row[1]['Roi.Y1']:row[1]['Roi.Y2'], :]
        # Resize về kích cỡ chuẩn
        img = cv2.resize(pixel, (64,64))

        # Thêm vào list dữ liệu
        pixels.append(img)

        # Thêm nhãn cho ảnh
        labels.append(row[1].ClassId)
print("Finish")

Finish


Trộn dữ liệu và chia dữ liệu thành 3 phần với tỉ lệ Train:Validation:Test = 6:2:2

In [13]:
# Chuẩn hoá dữ liệu pixels và labels
pixels = np.array(pixels)
labels = keras.utils.np_utils.to_categorical(labels)

# Nhào trộn dữ liệu ngẫu nhiên
randomize = np.arange(len(pixels))
np.random.shuffle(randomize)
X = pixels[randomize]
y = labels[randomize]

print("X = ",X.shape)

# Chia dữ liệu theo tỷ lệ 60% train và 40% còn lại cho val và test
train_size = int(X.shape[0] * 0.6)
X_train, X_val_test = X[:train_size], X[train_size:]
y_train, y_val_test = y[:train_size], y[train_size:]

val_size = int(X_val_test.shape[0] * 0.5) # 50% của phần 40% bên trên
X_val, X_test = X_val_test[:val_size], X_val_test[val_size:]
y_val, y_test = y_val_test[:val_size], y_val_test[val_size:]

print("X_train = ",X_train.shape)
print("X_val = ",X_val.shape)
print("X_test = ",X_test.shape)

X =  (39209, 64, 64, 3)
X_train =  (23525, 64, 64, 3)
X_val =  (7842, 64, 64, 3)
X_test =  (7842, 64, 64, 3)


Xây dựng model sử dụng Transfer Learning với mạng VGG16

In [17]:
input_shape=(64,64,3)
filter_size = (3,3)
pool_size = (2, 2)
output_size = 43
model = Sequential()

model.add(Conv2D(16, filter_size, activation="relu", padding="same", input_shape=(64, 64, 3)))
model.add(Conv2D(16, filter_size, activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2)))

model.add(Conv2D(32, filter_size, activation="relu", padding="same"))
model.add(Conv2D(32, filter_size, activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2)))

model.add(Conv2D(64, filter_size, activation="relu", padding="same"))
model.add(Conv2D(64, filter_size, activation="relu", padding="same"))
model.add(Conv2D(64, filter_size, activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2)))

model.add(Conv2D(128, filter_size, activation="relu", padding="same"))
model.add(Conv2D(128, filter_size, activation="relu", padding="same"))
model.add(Conv2D(128, filter_size, activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2)))

model.add(Conv2D(256, filter_size, activation="relu", padding="same"))
model.add(Conv2D(256, filter_size, activation="relu", padding="same"))
model.add(Conv2D(256, filter_size, activation="relu", padding="same"))
model.add(MaxPooling2D(pool_size=pool_size, strides=(2,2)))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))
model.add(Dense(output_size, activation='softmax'))
          
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=1e-4), metrics=['accuracy'])
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_39 (Conv2D)           (None, 64, 64, 16)        448       
_________________________________________________________________
conv2d_40 (Conv2D)           (None, 64, 64, 16)        2320      
_________________________________________________________________
max_pooling2d_15 (MaxPooling (None, 32, 32, 16)        0         
_________________________________________________________________
conv2d_41 (Conv2D)           (None, 32, 32, 32)        4640      
_________________________________________________________________
conv2d_42 (Conv2D)           (None, 32, 32, 32)        9248      
_________________________________________________________________
max_pooling2d_16 (MaxPooling (None, 16, 16, 32)        0         
_________________________________________________________________
conv2d_43 (Conv2D)           (None, 16, 16, 64)       

Train Model

In [19]:
model.fit(X_train, y_train, epochs=10, batch_size=16, validation_data=(X_val, y_val))

Epoch 1/10
  28/1471 [..............................] - ETA: 7:19 - loss: 0.0038 - accuracy: 0.9978

KeyboardInterrupt: 

Save model

In [20]:
model.save("traffic_sign_model.h5")

Load Model

In [21]:
model = keras.models.load_model("traffic_sign_model.h5")

Test Model

In [22]:
model.evaluate(X_test, y_test)

246/246 [==============================] - 17s 68ms/step - loss: 0.0424 - accuracy: 0.9892


[0.042423781007528305, 0.98916095495224]